In [7]:
pip install opencv-python opencv-python-headless tensorflow


   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   - -------------------------------------- 1.0/38.8 MB 5.6 MB/s eta 0:00:07
   -- ------------------------------------- 2.6/38.8 MB 7.2 MB/s eta 0:00:06
   --- ------------------------------------ 3.4/38.8 MB 6.1 MB/s eta 0:00:06
   ---- ----------------------------------- 3.9/38.8 MB 5.0 MB/s eta 0:00:07
   ---- ----------------------------------- 4.7/38.8 MB 5.4 MB/s eta 0:00:07
   ----- ---------------------------------- 5.5/38.8 MB 4.5 MB/s eta 0:00:08
   ------- -------------------------------- 6.8/38.8 MB 4.7 MB/s eta 0:00:07
   ------- -------------------------------- 7.3/38.8 MB 4.5 MB/s eta 0:00:08
   -------- ------------------------------- 8.1/38.8 MB 4.4 MB/s eta 0:00:07
   --------- ------------------------------ 8.9/38.8 MB 4.2 MB/s eta 0:00:08
   ---------- ----------------------------- 9.7/38.8 MB 4.2 MB/s eta 0:00:07
   ---------- ----------------------------- 10.2/38.8 MB 4.1 MB/s eta 0:00:07
   --

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\hp\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import sys

# Add the path to the SORT directory where sort.py is located
sys.path.append(r'C:\Users\hp\Desktop\Visual Computing\Sorting\sort')

# Import the Sort class from the SORT module
from sort import Sort

# Load YOLO
yolo_net = cv2.dnn.readNet(r'yolov3.weights', r'yolov3.cfg')
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo_net.getUnconnectedOutLayers()]
with open(r'C:\path\to\coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Load video
cap = cv2.VideoCapture(r'C:\Users\hp\Desktop\Visual Computing\Datasets\carv.mp4')

# Initialize SORT tracker and variables
tracker = Sort()
vehicle_count = 0
frame_count = 0
interval = 10  # Process every 10th frame for efficiency
counting_line_position = 300
tracked_vehicles = {}  # Track vehicle IDs that cross the counting line

# Create a figure for displaying the video with the count
fig, ax = plt.subplots()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % interval != 0:
        continue

    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    outs = yolo_net.forward(output_layers)

    # Prepare the detections
    detections = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 2:  # '2' is the class ID for vehicles (in COCO dataset)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                detections.append([x, y, x + w, y + h, confidence])

    # Update tracker
    tracked_objects = tracker.update(np.array(detections))

    for obj in tracked_objects:
        x1, y1, x2, y2, obj_id = map(int, obj)

        # Draw bounding box and ID label
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {obj_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Vehicle counting when the vehicle crosses the line
        vehicle_center_y = y1 + (y2 - y1) // 2
        if vehicle_center_y > counting_line_position and obj_id not in tracked_vehicles:
            vehicle_count += 1
            tracked_vehicles[obj_id] = True

    # Convert frame from BGR (OpenCV format) to RGB (Matplotlib format)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Display the frame using matplotlib
    ax.clear()
    ax.imshow(frame_rgb)
    ax.axis('off')
    ax.set_title(f"Vehicle Count: {vehicle_count}")
    plt.pause(0.01)  # Small pause to update the frame

# Release video capture
cap.release()
cv2.destroyAllWindows()
plt.show()

print(f"Total vehicle count: {vehicle_count}")


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'cv::dnn::dnn4_v20240521::readNetFromDarknet'
